In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import pycocotools.mask as mask_util
import shutil

In [ ]:
# Import py360convert from lib folder
import sys
sys.path.insert(0, '/home/lombardo/sidewalk-accessibility-features/lib/py360convert')

import py360convert

In [ ]:
def map_faces(index):
    # Function that maps the face index to the corresponding face
    # Mapping: 0F 1R 2B 3L 4U 5D
    # For example: index == 0, return 'front'
    if index == 0:
        return 'front'
    elif index == 1:
        return 'right'
    elif index == 2:
        return 'back'
    elif index == 3:
        return 'left'
    elif index == 4:
        return 'top'
    elif index == 5:
        return 'bottom'

In [ ]:
def reverse_map_faces(face):
    # Function that maps the face name to the corresponding index
    # Mapping: 'front' -> 0, 'right' -> 1, 'back' -> 2, 'left' -> 3, 'top' -> 4, 'bottom' -> 5
    face_mapping = {'front': 0, 'right': 1, 'back': 2, 'left': 3, 'top': 4, 'bottom': 5}
    return face_mapping.get(face)

In [ ]:
pano_id_example = '9VW3AUbKBdSPqnURnwRSLQ'
reoriented_path = os.path.join('/var/scratch/lombardo/download_PS/reoriented', f'{pano_id_example}.jpg')

# Open image
img = cv2.imread(reoriented_path)
# Convert to RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Open labels_reprojected.csv
labels_reprojected_path = '/var/scratch/lombardo/download_PS/labels_1024_reprojected.csv'

# Test with a single pano_id
labels_reprojected = pd.read_csv(labels_reprojected_path)

# labels_reprojected contains (reoriented_pano_x,reoriented_pano_y) coordinates. For each label with 'gsv_panorama_id' == pano_id_example,
# we want to retrieve the corresponding (reoriented_pano_x,reoriented_pano_y) coordinates and visualize them on the reoriented image.
gt_points = []
for index, row in labels_reprojected.iterrows():
    if row['gsv_panorama_id'] == pano_id_example:
        # Retrieve coordinates
        image_width = row['pano_width']
        image_height = row['pano_height']
        sv_image_x = row['pano_x']
        sv_image_y = row['pano_y']
        print(f'')
        print(f'Coordinates before scaling: {sv_image_y},{sv_image_x}')
        scaled_x = row['scaled_pano_x']
        scaled_y = row['scaled_pano_y']
        reoriented_pano_x = row['reoriented_pano_x']
        reoriented_pano_y = row['reoriented_pano_y']
        print(f'Coordinates before reorientation: {scaled_y},{scaled_x}')
        print(f'Coordinates after reorientation: {reoriented_pano_y},{reoriented_pano_x}')
        
        # Retrieve coordinates
        #reoriented_pano_x = row['reoriented_pano_x']
        #reoriented_pano_y = row['reoriented_pano_y']
        reprojected_pano_x = row['reprojected_pano_x']
        reprojected_pano_y = row['reprojected_pano_y']
        face_idx = row['face_idx']
        print(f'Face idx: {face_idx}')
        print(f'Coordinates after reprojection: {reprojected_pano_y},{reprojected_pano_x}')

        gt_points.append([face_idx, (reprojected_pano_y, reprojected_pano_x)])

print(gt_points)

# Initialize a dictionary to store the subplots for each face
subplots = {}

# Visualize the points
for point in gt_points:
    face = map_faces(point[0])
    print(f'Face: {face}')
    
    if face not in subplots:
        # Create a new subplot for the face
        subplots[face] = plt.figure(figsize=(15, 5))

    mask_path = f'/var/scratch/lombardo/download_PS/masks/{pano_id_example}/{face}.png'
    path = f'/var/scratch/lombardo/download_PS/reprojected/{pano_id_example}/{face}.png'
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    # Resize mask to match image size
    mask = cv2.resize(mask, (img.shape[1], img.shape[0]))
    
    # Threshold the image
    _, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    # Apply connected components with statistics
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)

    print('Number of connected components: ' + str(num_labels - 1))

    # Create an empty image to visualize the connected components
    output = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    # Assign a unique color for each connected component
    for i in range(1, num_labels):
        color = (np.random.randint(0, 256), np.random.randint(0, 256), np.random.randint(0, 256))
        output[labels == i] = color

        # Get the area of the connected component from the stats array
        area = stats[i, cv2.CC_STAT_AREA]

        #print(f"Component {i}: Area = {area}, Color = {color}")

    # Get the subplot and plot img, output, and the point
    plt.figure(subplots[face].number)
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.imshow(output, alpha=0.4)
    plt.scatter(point[1][1], point[1][0], color='red', s=7)

# Show all the subplots
for face, subplot in subplots.items():
    plt.figure(subplot.number)
    plt.show()


'''/var/scratch/lombardo/download_PS/reoriented/6mRBTtpLi7Bi3Tgu_fxOtA.jpg
Coordinates before reorientation: 699,1743
Coordinates after reorientation: 699,140
Coordinates after reprojection: 877,276
[[2, (877, 276)]]'''

In [ ]:
img_example_path = os.path.join('/var/scratch/lombardo/download_PS/reoriented', f'{pano_id_example}.jpg')
print(img_example_path)

# Open image
img = cv2.imread(img_example_path)
# Convert to RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print(img.shape)
# Show image
# Figure: medium size
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.scatter(1459,590,color='red', s=15)
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load image from 'res/equirectangular.png'
img_path = 'res/equirectangular.png'
example_pano = 'res/example_pano.png'
img = cv2.imread(example_pano)
# Convert to RGB
#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Draw a red point at coordinates (400,800)
cv2.circle(img, (500,680), 10, (0,0,255), -1)

# Save image as example_pano_triangle.png
cv2.imwrite('res/example_pano_point.png', img)

# Convert to cubemap
cubemap = py360convert.e2c(img, face_w=1024)
# Print dtype of cubemap
print(cubemap.dtype)
# Flip horizontally the images of cubemap at indices 0 and 2
#cubemap[[0,2]] = cubemap[[0,2]][:,::-1]
# Save cubemap as example_pano_triangle_cb.png
cv2.imwrite('res/example_pano_point_cb.png', cubemap)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.gridspec as gridspec

# Assuming you have 5 images and their corresponding label qualities
img1 = plt.imread('res/problems_clustering/label_quality/visualize__-7gKmiV-npQwLy_CD3v7Q_back.png')
img2 = plt.imread('res/problems_clustering/label_quality/visualize__A3VGaMhL4XcGQxtFUiAfA_back.png')
img3 = plt.imread('res/problems_clustering/label_quality/visualize__Nkwp741bbV44txP6PRo1A_right.png')
img4 = plt.imread('res/problems_clustering/label_quality/visualize_-RHS-ZYDt0sCDy2aS4AwOQ_right.png')
img5 = plt.imread('res/problems_clustering/label_quality/visualize_0sAG8-ozIw7wANw_hsTCHA_back.png')
img6 = plt.imread('res/problems_clustering/label_not_found/visualize__WZ_Q2QCOe9NEypucJnClQ_left.png')
img7 = plt.imread('res/problems_clustering/label_not_found/visualize_-juZj6rt9x1unIRBa0D7fQ_right.png')
img8 = plt.imread('res/problems_clustering/label_not_found/visualize_-qi4eywPKVd0sHlHqcWVkA_left.png')
img9 = plt.imread('res/problems_clustering/label_not_found/visualize_0E_M8eWvyACpG_fUq4TFrQ_left.png')
img10 = plt.imread('res/problems_clustering/label_not_found/visualize_0sKrgbDBfDrizWjCnXI0FQ_right.png')
img11 = plt.imread('res/problems_clustering/all_labels_not_found/visualize__pjTmksYX5Vpgoqm_GSOKA_right.png')
img12 = plt.imread('res/problems_clustering/all_labels_not_found/visualize__ZuTw7GC38rShAUjL-dLmQ_left.png')
img13 = plt.imread('res/problems_clustering/all_labels_not_found/visualize_-ELBA4Cjr_WBoetht0GVbQ_right.png')
img14 = plt.imread('res/problems_clustering/all_labels_not_found/visualize_-RHS-ZYDt0sCDy2aS4AwOQ_left.png')
img15 = plt.imread('res/problems_clustering/all_labels_not_found/visualize_0wLMkTJ7IaDz8mJm3uNeHg_front.png')
img16 = plt.imread('res/problems_clustering/not_relevant_objects/visualize_-CD3qZk9zyeumBU_w7Yl5A_front.png')
img17 = plt.imread('res/problems_clustering/not_relevant_objects/visualize_-rU6BN-wNeRclahZxn2nug_back.png')
img18 = plt.imread('res/problems_clustering/not_relevant_objects/visualize_0h2CfQQ_Ca5sRgU5Zze01g_right.png')
img19 = plt.imread('res/problems_clustering/not_relevant_objects/visualize_0NjLmtmwNh95YlOyyydtnA_front.png')
img20 = plt.imread('res/problems_clustering/not_relevant_objects/visualize_0vrKe1xEAZkxUXYhtwRtaQ_front.png')
img21 = plt.imread('res/problems_clustering/multiple-objects_mask/visualize__ZuTw7GC38rShAUjL-dLmQ_front.png')
img22 = plt.imread('res/problems_clustering/multiple-objects_mask/visualize_-ay7zopCWOrSjeUuirk91A_right.png')
img23 = plt.imread('res/problems_clustering/multiple-objects_mask/visualize_-WI1nKjShmeFVRGsrFsFpw_front.png')
img24 = plt.imread('res/problems_clustering/multiple-objects_mask/visualize_0FHr9qesIVgabJ8hR7Uf2A_back.png')
img25 = plt.imread('res/problems_clustering/multiple-objects_mask/visualize_1GIXHFehrd1Xbu2JbmzlFw_left.png')
img26 = plt.imread('res/problems_clustering/noisy_masks/visualize_2fPjt5siCmoOpY1kMe3t3w_left.png')
img27 = plt.imread('res/problems_clustering/noisy_masks/visualize_2iKOp2S2_K4WrxfLS1jDlA_right.png')
img28 = plt.imread('res/problems_clustering/noisy_masks/visualize_2qShzHuplxE5Zp02V3Fnrw_front.png')
img29 = plt.imread('res/problems_clustering/noisy_masks/visualize_2Yg6fuCTqr_5Uc5IcGPs4w_front.png')
img30 = plt.imread('res/problems_clustering/noisy_masks/visualize_3iQ5g0gAW8uyiTwiRNytaQ_back.png')
images = [img1, img2, img3, img4, img5, img6, img7, \
            img8, img9, img10, img11, img12, img13, \
            img14, img15, img16, img17, img18, img19, \
            img20, img21, img22, img23, img24, img25, \
            img26, img27, img28, img29, img30]
label_qualities = [4, 3, 5, 2, 4] * 6  # Placeholder label qualities

In [ ]:
# Create images made of two rows and five random images
# images = np.random.randint(0, 255, size=(10, 256, 256, 3), dtype=np.uint8)

# Define the y-axis labels for each row
y_labels = ['Label quality', 'Label not found', 'All labels not found', 'Not relevant objects', \
            'Multiple-objects masks', 'Noisy masks']

# Define grid
gs = gridspec.GridSpec(6, 6, width_ratios=[1, 5, 5, 5, 5, 5]) 

fig = plt.figure(figsize=(12, 14))

# Iterate over each row and subplot, plot the image, and set the y-axis label
for i in range(6):
    ax = plt.subplot(gs[i, 0])
    ax.text(0.5, 0.5, y_labels[i], ha='center', va='center', rotation='vertical', fontsize=10)
    ax.axis('off')

    for j in range(5):
        ax = plt.subplot(gs[i, j+1])
        ax.imshow(images[i * 5 + j])  # Plot the image
        ax.axis('off')  # Turn off the axis for cleaner display

# Adjust the spacing between subplots and display the plot
plt.subplots_adjust(wspace=0.05, hspace=0.05)
plt.show()

# Save the figure
#fig.savefig('res/problems_clustering/plot.png', bbox_inches='tight')

'''# Create subplots for each image
fig, axs = plt.subplots(6, 5, figsize=(12, 14))

# Define the y-axis labels for each row
y_labels = ['Label quality', 'Label not found', 'All labels not found', 'Not relevant objects', \
            'Multiple-objects masks', 'Noisy masks']

# Iterate over each row and subplot, plot the image, and set the y-axis label
for i, row in enumerate(axs):
    for j, ax in enumerate(row):
        ax.imshow(images[i * 5 + j])  # Plot the image
        ax.set_axis_off()  # Turn off the axis for cleaner display
        
        # Add the vertical text label only for the first subplot in each row
        if j == 0:
            ax.text(-10, -10, y_labels[i], rotation='vertical', va='top', ha='right', fontsize=10)

#plt.tight_layout()
# Adjust the spacing between subplots and display the plot
plt.subplots_adjust(wspace=0.05, hspace=0.05)
plt.show()
# Save the figure'''

fig.savefig('res/problems_clustering/plot.png', bbox_inches='tight')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import re

# Load your data
df1 = pd.read_csv('res/problems_clustering_baseline.csv')
df2 = pd.read_csv('res/problems_clustering_context.csv')

# Define labels
#labels = ['Label quality', 'Label not found', 'All labels not found', 'Not relevant objects', 'Multiple-objects masks', 'Noisy masks']

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

def count_labels(df, column_name):
    # Initialize a counter
    counter = Counter()
    
    # Iterate over the rows of the dataframe
    for labels in df[column_name]:
        # Skip NA values
        if pd.notnull(labels):
            # Split the string into labels and strip whitespace
            labels = [label.strip().lower() for label in labels.split(',')]
            
            # Count the labels
            counter.update(labels)
    
    return counter

# Count the labels in the 'problem' column
counter1 = count_labels(df1, 'problem')
counter2 = count_labels(df2, 'problem')

# Convert the counters to dataframes
df1_counts = pd.DataFrame.from_dict(counter1, orient='index', columns=['Baseline'])
df2_counts = pd.DataFrame.from_dict(counter2, orient='index', columns=['Baseline + Context'])

# Merge the dataframes
df_counts = df1_counts.join(df2_counts, how='outer').fillna(0)

# Reset index to make 'index' a column (for seaborn)
df_counts.reset_index(level=0, inplace=True)

# Rename the 'index' column to 'labels'
df_counts.rename(columns={'index': 'labels'}, inplace=True)

# Capitalize the first letter of each label
df_counts['labels'] = df_counts['labels'].str.capitalize()

# Melt the dataframe to make it suitable for seaborn
df_counts = df_counts.melt('labels', var_name='Experiments', value_name='counts')

# Plot the data
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='labels', y='counts', hue='Experiments', data=df_counts)
plt.title('Counts of problems in clusters')
plt.ylabel('Count')
plt.xlabel('Problem categories')
plt.xticks(rotation=45)

# Add count numbers as annotations above the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.0f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 5), textcoords='offset points')

plt.show()

# Save in high quality
fig = ax.get_figure()
fig.savefig('res/problems_clustering/plot2.png', bbox_inches='tight', dpi=300)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2

img1_path = 'res/beforeafter/before/visualize_1GYNc35brG1DJCtFYWs8KA_right.png'
img2_path = 'res/beforeafter/before/visualize_1VEVLWYfmi0vVyEsXoJD8A_right.png'
img3_path = 'res/beforeafter/before/visualize_2atLrFYpUNAyUc7f7M8mzA_back.png'
img4_path = 'res/beforeafter/before/visualize_2E3r3TggV5YhVAYv1q0euA_right.png'
img5_path = 'res/beforeafter/before/visualize_2ke_0eGW8-nEpc1cFyBM2A_front.png'
img6_path = 'res/beforeafter/after/visualize_1GYNc35brG1DJCtFYWs8KA_right.png'
img7_path = 'res/beforeafter/after/visualize_1VEVLWYfmi0vVyEsXoJD8A_right.png'
img8_path = 'res/beforeafter/after/visualize_2atLrFYpUNAyUc7f7M8mzA_back.png'
img9_path = 'res/beforeafter/after/visualize_2E3r3TggV5YhVAYv1q0euA_right.png'
img10_path = 'res/beforeafter/after/visualize_2ke_0eGW8-nEpc1cFyBM2A_front.png'

# Load images
img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)
img3 = cv2.imread(img3_path)
img4 = cv2.imread(img4_path)
img5 = cv2.imread(img5_path)
img6 = cv2.imread(img6_path)
img7 = cv2.imread(img7_path)
img8 = cv2.imread(img8_path)
img9 = cv2.imread(img9_path)
img10 = cv2.imread(img10_path)

imgs_baseline = [img1, img2, img3, img4, img5]
imgs_context = [img6, img7, img8, img9, img10]

# Convert the images in imgs_baseline and imgs_context to RGB
imgs_baseline = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in imgs_baseline]
imgs_context = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in imgs_context]

# Define grid
gs = gridspec.GridSpec(2, 6, width_ratios=[2, 5, 5, 5, 5, 5]) 

fig = plt.figure(figsize=(10, 4))

# Set up class names
class_names = ['Baseline', 'Prefiltering']
for i in range(2):
    ax = plt.subplot(gs[i, 0])
    ax.text(0.5, 0.5, class_names[i], ha='center', va='center', rotation='vertical', fontsize=9)
    ax.axis('off')

# Display baseline images
for i in range(5):
    ax = plt.subplot(gs[0, i+1])
    ax.imshow(imgs_baseline[i])
    ax.axis('off')

# Display context images
for i in range(5):
    ax = plt.subplot(gs[1, i+1])
    ax.imshow(imgs_context[i])
    ax.axis('off')

plt.subplots_adjust(wspace=0.07, hspace=0.05)

#plt.tight_layout()
plt.show()

# Save in high quality in res/beforeafter
fig.savefig('res/beforeafter/plot_revised.png', bbox_inches='tight', dpi=300)

In [ ]:
'''Visualization of Precision faulty images'''
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2

img1_path = 'res/precision_faulty/visualize_1GYNc35brG1DJCtFYWs8KA_right.png'
img2_path = 'res/precision_faulty/visualize_0wLMkTJ7IaDz8mJm3uNeHg_right.png'
img3_path = 'res/precision_faulty/visualize_-CD3qZk9zyeumBU_w7Yl5A_front.png'
img4_path = 'res/precision_faulty/visualize_-ay7zopCWOrSjeUuirk91A_right.png'
img5_path = 'res/precision_faulty/visualize__pjTmksYX5Vpgoqm_GSOKA_back.png'

# Load images
img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)
img3 = cv2.imread(img3_path)
img4 = cv2.imread(img4_path)
img5 = cv2.imread(img5_path)

imgs = [img1, img2, img3, img4, img5]

# Convert the images in imgs_baseline and imgs_context to RGB
imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in imgs]

# Define grid
gs = gridspec.GridSpec(1, 5) 

fig = plt.figure(figsize=(10, 4))

# Display baseline images
for i, img in enumerate(imgs):
    ax = plt.subplot(gs[0, i])
    ax.imshow(img)
    ax.axis('off')

plt.subplots_adjust(wspace=0.07, hspace=0.05)

#plt.tight_layout()
plt.show()

# Save in high quality in res/beforeafter
fig.savefig('res/precision_faulty/plot.png', bbox_inches='tight', dpi=300)

In [ ]:
import os
import json
import requests
import argparse
import geopandas as gpd

def get_ps_labels():

    base_url = "https://sidewalk-amsterdam.cs.washington.edu/v2/access/attributesWithLabels?lat1={}&lng1={}&lat2={}&lng2={}" 
    
    whole = (52.303, 4.8, 52.425, 5.05)
    centrum_west = (52.364925, 4.87444, 52.388692, 4.90641)
    test = (52.0, 4.0, 53.0, 5.0)

    coords = test

    url = base_url.format(*coords)

    #label_dump = os.path.join(args.label_dump, 'attributesWithLabels')

    try:
        project_sidewalk_labels = json.load(open('res/attributesWithLabels', 'r'))
    except Exception as e:
        print("Couldn't load local dump")
        project_sidewalk_labels = requests.get(url.format(*coords)).json()
        #json.dump(project_sidewalk_labels, open(label_dump, 'w'))

    ps_labels_df = gpd.GeoDataFrame.from_features(project_sidewalk_labels['features'])
    # Print length before filtering
    print('Length of labels from attributesWithLabels API: ', len(ps_labels_df))

    return ps_labels_df

def get_xy_coords_ps_labels():
    # Send a get call to this API: https://sidewalk-amsterdam-test.cs.washington.edu/adminapi/labels/cvMetadata
    other_labels = requests.get('https://sidewalk-amsterdam.cs.washington.edu/adminapi/labels/cvMetadata').json()
    other_labels_df = pd.DataFrame(other_labels)
    print('Length raw labels from cvMetadata API (low quality) : ', len(other_labels_df))

    return other_labels_df

def visualize_labels(image_path, gt_points):
    # Load the image
    # Depending on the machine, the path might be different
    # Try to load the image with and without the .jpg extension
    try:
        image = plt.imread(image_path + '.jpg')
    except:
        image = plt.imread(image_path)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.imshow(image, cmap='gray')

    for gt_point in gt_points:
        ax.scatter(gt_point[1], gt_point[0], color='red', s=7)

    # Set small tick labels
    ax.tick_params(axis='both', which='both', labelsize=6)
    # No ticks
    ax.set_xticks([])

    # Add legend for red points
    ax.scatter([], [], color='red', s=8, label='Ground Truth')
    ax.legend(prop={'size': 5})

    # Remove axes
    ax.axis('off')

    # Save the image
    fig.savefig('res/example_pano_labels.png', dpi=300, bbox_inches='tight')
    # Close the figure
    plt.close(fig)

def compute_label_coordinates(image_path, dataframe, pano_id):            

    # Load panorama image
    # Depending on the machine, the path might be different
    # Try to load the image with and without the .jpg extension
    try:
        image = plt.imread(image_path + '.jpg')
    except:
        image = plt.imread(image_path)

    # Find all the labels in the panorama
    labels = dataframe[dataframe['gsv_panorama_id'] == pano_id]

    labels_coords = []

    # For each label, find its coordinates in the panorama
    for index, row in labels.iterrows():
        #print('Labels coming from panorama:', row['gsv_panorama_id'])
        label_name = row['label_id']
        image_width = row['pano_width']
        image_height = row['pano_height']
        sv_image_x = row['pano_x']
        # Momentarily change the definition of sv_image_y. For now,
        # sv_image_y is the offset w.r.t. the middle of the image
        # which means that the real y coordinate is y = (image_width / 2) - sv_image_y
        sv_image_y = row['pano_y']
        #real_y = (image_height / 2) - sv_image_y

        #print(f'Original image width: {image_width}, height: {image_height}')
        #print(f'Original label {label_name} coordinates: {sv_image_x}, {sv_image_y}')
        #print(f'Original label {label_name} with real y-coordinates' \
        #    f'({image_height}/2){sv_image_y}: {sv_image_x}, {real_y}')

        # image_width : sv_image_x = my_pano_width : sv_pano_x
        # sv_pano_x = (sv_image_x * my_pano_width) / image_width
        sv_pano_x = int(sv_image_x*image.shape[1]/image_width)
        sv_pano_y = int(sv_image_y*image.shape[0]/image_height)

        #print(f'Scaled label {label_name} coordinates: {sv_pano_x}, {sv_pano_y} \n')

        # Visualize the point (sv_pano_x, sv_pano_y) on the image
        #print(f'Label {label_name} coordinates: {sv_pano_x}, {sv_pano_y} \n')
        #plt.scatter(sv_pano_x, sv_pano_y, color='red', s=10)

        labels_coords.append((sv_pano_y, sv_pano_x))

    return labels_coords

def reorient_point(point, img_size, heading):
    # We select a tolerance of 0.1 degrees so that 
    # the point is not reoriented if the heading is close to 0
    tolerance = 0.1
    if abs(heading) <= tolerance:
        return point

    else:
        # Reshift point according to heading
        shift = heading / 360
        pixel_split = int(img_size[0] * shift)
        
        y, x = point
        new_x = (x - pixel_split) % img_size[0]

        reoriented_point = (y, new_x)
        
        return reoriented_point

In [ ]:
import pandas as pd

# Get Project Sidewalk labels from API
ps_labels_df = get_ps_labels()
# Get the labels from another API endpoint
other_labels_df = get_xy_coords_ps_labels()

# Filter other_labels_df to only contain obstacles ('label_type_id' == 3)
other_labels_df = other_labels_df[other_labels_df['label_type_id'] == 3]

print(f'Number of obstacles labels from cvMetadata API (low quality): {len(other_labels_df)}')

# Intersect other_labels_df with ps_labels_df
other_labels_df = other_labels_df[other_labels_df['label_id'].isin(ps_labels_df['label_id'])]
print(f'Number of obstacles labels in other_labels_df after filtering for high quality data: {len(other_labels_df)}')

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Print the labels associated with 'pano_id' value: -RHS-ZYDt0sCDy2aS4AwOQ
labels = other_labels_df[other_labels_df['gsv_panorama_id'] == '-RHS-ZYDt0sCDy2aS4AwOQ']

# Load image
img_path = 'res/-RHS-ZYDt0sCDy2aS4AwOQ.jpg'
img = cv2.imread(img_path)
# Take the size of the image
img_size = img.shape[:2]

heading = 251.48272705078125

# Compute the coordinate of the labels in the image
labels_coords = compute_label_coordinates(img_path, other_labels_df, '-RHS-ZYDt0sCDy2aS4AwOQ')

reoriented_labels_coords = []
for label in labels_coords:
    print('Initial coordinates of the label:', label)
    # Reorient the point according to the heading
    new_coords = reorient_point(label, img_size, heading)
    print('New coordinates: ', new_coords)
    reoriented_labels_coords.append(new_coords)

visualize_labels(img_path, labels_coords)
    

In [ ]:
import numpy as np

# Create some fake masks of random shapes
mask1 = np.zeros((1024, 1024), dtype=np.uint8)
mask1[100:200, 100:200] = 255
mask2 = np.zeros((1024, 1024), dtype=np.uint8)
mask2[300:400, 300:400] = 255
mask3 = np.zeros((1024, 1024), dtype=np.uint8)
mask3[500:600, 500:600] = 255

# Create areas and centroids for the masks based on the fake masks
area1 = np.sum(mask1 == 255)
area2 = np.sum(mask2 == 255)
area3 = np.sum(mask3 == 255)

centroid1 = np.array([150, 150])
centroid2 = np.array([350, 350])
centroid3 = np.array([550, 550])

# Create a fake bbox (inside an image of size 1024x1024)
fake_bbox = [5, 10, 150, 250]

# Visualize the bbox and the masks on the black image
img = np.zeros((1024, 1024), dtype=np.uint8)
img[fake_bbox[0]:fake_bbox[2], fake_bbox[1]:fake_bbox[3]] = 255

import matplotlib.pyplot as plt
# Plot the bbox and the masks on the same image (only one axis)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(img, cmap='gray')
ax.imshow(mask1, cmap='jet', alpha=0.5)
ax.imshow(mask2, cmap='jet', alpha=0.5)
ax.imshow(mask3, cmap='jet', alpha=0.5)
# Plot the centroids
ax.scatter(centroid1[1], centroid1[0], color='red', s=10)
ax.scatter(centroid2[1], centroid2[0], color='red', s=10)
ax.scatter(centroid3[1], centroid3[0], color='red', s=10)
plt.show()


# Create a fake dictionary with the masks, areas, and centroids
panos_info = {
    'pano1': {
        'face1': {
            'masks': [mask1, mask2],
            'areas': [area1, area2],
            'centroids': [centroid1, centroid2]
        },
        'face2': {
            'masks': [mask3],
            'areas': [area3],
            'centroids': [centroid3]
        }
    }
}

data = [{ "value": 0, "label": "background"}, {"value": 1, "label": "sidewalk", "logit": 0.9, "box": fake_bbox}]

# Create a new panos_info dictionary with the same structure as the original one
new_panos_info = {}
for pano_id in panos_info:
    new_panos_info[pano_id] = {}
    for face_idx in panos_info[pano_id]:
        new_panos_info[pano_id][face_idx] = {
            'masks': [],
            'areas': [],
            'centroids': []
        }

for pano_id in panos_info: 
    for face_idx in panos_info[pano_id]:
        # Iterate over each mask in panos_info[pano_id][face_idx]['masks']
        for mask, area, centroid in zip(panos_info[pano_id][face_idx]['masks'], panos_info[pano_id][face_idx]['areas'], panos_info[pano_id][face_idx]['centroids']):
            # Iterate over each bounding box in data that has the label 'sidewalk'
            # If there is no 'sidewalk' label, look for 'sidewalkroad' label
            # If there are no 'sidewalk' or 'sidewalkroad' labels, skip the current face index
            for d in data:
                # d has the following structure: {"value", "label", "logit", "box": []}
                # Iterate over the dictionary keys
                if d.get('label') == 'sidewalk' or 'sidewalkroad':
                    bbox = d.get('box')
                    if bbox is None:
                        continue
                    bbox_mask = np.zeros((mask.shape))
                    bbox_mask[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])] = 255
                    if np.any(np.logical_and(mask, bbox_mask)):
                        # Append the new mask, area, and centroid to the corresponding lists in the new_panos_info dictionary
                        new_panos_info[pano_id][face_idx]['masks'].append(mask)
                        new_panos_info[pano_id][face_idx]['areas'].append(area)
                        new_panos_info[pano_id][face_idx]['centroids'].append(centroid)

# Visualize the new masks
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(img, cmap='gray')
for face_idx in new_panos_info['pano1']:
    for mask, centroid in zip(new_panos_info['pano1'][face_idx]['masks'], new_panos_info['pano1'][face_idx]['centroids']):
        ax.imshow(mask, cmap='jet', alpha=0.5)
        ax.scatter(centroid[1], centroid[0], color='red', s=10)

In [ ]:
import numpy as np
import cv2
from scipy.spatial.distance import cdist

# Create some fake masks of random shapes
mask1 = np.zeros((1024, 1024), dtype=np.uint8)
mask1[100:200, 100:200] = 255
mask2 = np.zeros((1024, 1024), dtype=np.uint8)
mask2[300:400, 300:400] = 255
mask3 = np.zeros((1024, 1024), dtype=np.uint8)
mask3[700:900, 700:900] = 255

# Create an empty image
img = np.zeros((1024, 1024), dtype=np.uint8)

# Create areas and centroids for the masks based on the fake masks
area1 = np.sum(mask1 == 255)
area2 = np.sum(mask2 == 255)
area3 = np.sum(mask3 == 255)

centroid1 = np.array([150, 150])
centroid2 = np.array([350, 350])
centroid3 = np.array([800, 800])

# Create a big fake mask (inside an image of size 1024x1024)
fake_mask = np.zeros((1024, 1024), dtype=np.uint8)
fake_mask[100:600, 100:600] = 255

import matplotlib.pyplot as plt
# Plot the bbox and the masks on the same image (only one axis)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(img, cmap='gray')
ax.imshow(fake_mask, cmap='jet', alpha=0.8)
ax.imshow(mask1, cmap='jet', alpha=0.5)
ax.imshow(mask2, cmap='jet', alpha=0.5)
ax.imshow(mask3, cmap='jet', alpha=0.5)
# Plot the centroids
ax.scatter(centroid1[1], centroid1[0], color='red', s=10)
ax.scatter(centroid2[1], centroid2[0], color='red', s=10)
ax.scatter(centroid3[1], centroid3[0], color='red', s=10)
plt.show()

# Create a fake dictionary with the masks, areas, and centroids
panos_info = {
    'pano1': {
        'face1': {
            'masks': [mask1, mask2],
            'areas': [area1, area2],
            'centroids': [centroid1, centroid2]
        },
        'face2': {
            'masks': [mask3],
            'areas': [area3],
            'centroids': [centroid3]
        }
    }
}

# Create a new panos_info dictionary with the same structure as the original one
new_panos_info = {}
for pano_id in panos_info:
    new_panos_info[pano_id] = {}
    for face_idx in panos_info[pano_id]:
        new_panos_info[pano_id][face_idx] = {
            'masks': [],
            'areas': [],
            'centroids': []
        }

for pano_id in panos_info: 
    for face_idx in panos_info[pano_id]:
        # Iterate over each mask in panos_info[pano_id][face_idx]['masks']
        for mask, area, centroid in zip(panos_info[pano_id][face_idx]['masks'], panos_info[pano_id][face_idx]['areas'], panos_info[pano_id][face_idx]['centroids']):
            print(f'Processing mask with area {area}')
            # Compute the minimum distance between the centroid of the mask and the fake_mask
            min_distance = np.min(cdist([centroid], np.argwhere(fake_mask == 255)))
            
            if min_distance < 50:
                # Append the new mask, area, and centroid to the corresponding lists in the new_panos_info dictionary
                new_panos_info[pano_id][face_idx]['masks'].append(mask)
                new_panos_info[pano_id][face_idx]['areas'].append(area)
                new_panos_info[pano_id][face_idx]['centroids'].append(centroid)
            else:
                # If there are no 'sidewalk' or 'sidewalkroad' labels, copy
                # the masks, areas, and centroids from panos_info to new_panos_info
                print('No sidewalk or sidewalkroad labels found. Skipping...')
                #new_panos_info[pano_id][face_idx]['masks'] = panos_info[pano_id][face_idx]['masks']
                #new_panos_info[pano_id][face_idx]['areas'] = panos_info[pano_id][face_idx]['areas']
                #new_panos_info[pano_id][face_idx]['centroids'] = panos_info[pano_id][face_idx]['centroids']

# Visualize the new masks
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(img, cmap='gray')
for face_idx in new_panos_info['pano1']:
    for mask, centroid in zip(new_panos_info['pano1'][face_idx]['masks'], new_panos_info['pano1'][face_idx]['centroids']):
        ax.imshow(mask, cmap='jet', alpha=0.5)
        ax.scatter(centroid[1], centroid[0], color='red', s=10)

In [ ]:
def scale(original_size, new_size):
    # Function that calculates the scaling factor to resize the masks
    return new_size / original_size

In [ ]:
import json
from matplotlib import patches

# Load pano
pano_path = '/var/scratch/lombardo/download_PS/reprojected/KE2eBXFqUMUn49kvWjsCqA/left.png'
pano = cv2.imread(pano_path, cv2.IMREAD_COLOR)
pano = cv2.cvtColor(pano, cv2.COLOR_BGR2RGB)
# Resize pano to 2310
#pano = cv2.resize(pano, (2310, 2310))
original_size = 2310
new_size = 1024
# Visualize pano
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(pano)


# Load bbox
bbox_path = '/var/scratch/lombardo/download_PS/segmented_B_0.25/KE2eBXFqUMUn49kvWjsCqA/mask_left.json'
with open(bbox_path, 'r') as f:
    data = json.load(f)

# Initialize bbox_mask
bbox_mask = np.zeros((new_size, new_size))

# Iterate over each bounding box in data that has the label 'sidewalk'
# If there is no 'sidewalk' label, look for 'sidewalkroad' label
# If there are no 'sidewalk' or 'sidewalkroad' labels, skip the current face index
for d in data:
    #print(f'Label: {d.get("label")}')
    # d has the following structure: {"value", "label", "logit", "box": []}
    # Iterate over the dictionary keys
    if d.get('label') in ['sidewalk', 'sidewalkroad']:
        #print(f'bbox: {d.get("box")}')
        sidewalk_found = True
        bbox = d.get('box')

        # Scale the bbox
        #scale_factor = scale(args.sidewalk_mask_size, args.size)
        #bbox = [x * scale_factor for x in bbox]

        #print(f'bbox: {bbox}')

        # Add the current bbox to the bbox_mask
        bbox_mask[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])] = 255

centroid = [1000.80615098, 1008.17183131]
# Dilate the centroid to make a mask of radius 10
centroid_mask = np.zeros((new_size, new_size))
centroid_mask[int(centroid[1]) - 10:int(centroid[1]) + 10, int(centroid[0]) - 10:int(centroid[0]) + 10] = 255

intersection = cv2.bitwise_and(centroid_mask, bbox_mask)
if np.count_nonzero(intersection) > 0:
    print('There is an intersection')
else:
    print('There is no intersection')

# Visualize bbox_mask on top of pano
ax.imshow(bbox_mask, cmap='jet', alpha=0.5)
ax.imshow(centroid_mask, cmap='jet', alpha=0.5)
ax.scatter(centroid[1], centroid[0], color='red', s=10)
plt.show()

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist
# Load mask
mask_path = '/var/scratch/lombardo/download_PS/segmented_B_0.25/-2uB6480IX2sxmwnAnlLiA/sidewalk_mask_back.jpg'
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
# Resize mask to 1024x1024
mask = cv2.resize(mask, (1024, 1024))

# Make the mask binary (0 or 255) using a threshold of 127
# Threshold the image
_, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

centroid = [550.80615098, 908.17183131]

# Compute the minimum distance between centroid and mask 
centroid = np.array(centroid)

# Reshape the centroid array to have two dimensions
array_centroid = np.array(centroid).reshape(1, -1)

# Compute the minimum distance between the centroid of the mask and the fake_mask
min_distance = np.min(cdist(array_centroid, np.argwhere(mask == 255)))

print(min_distance)


# Show both centroid and mask in a single plot, along with the distance (min_distance)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(mask, cmap='gray')
ax.scatter(centroid[1], centroid[0], color='red', s=10)
ax.set_title(f'Min distance: {min_distance}')
plt.show()

In [ ]:
import cv2
import matplotlib.pyplot as plt

mask_path = '/var/scratch/lombardo/download_PS/segmented_B_0.25/MmG7bzbVlHJDCIuMNinV9g/sidewalkroad_mask_right.jpg'
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
# Resize mask to 1024x1024
mask = cv2.resize(mask, (1024, 1024))
# Visualize
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(mask, cmap='gray')

# Visualization of bboxes and masks

In [ ]:
# Load input_coco_format .json file 
import json
#json_path = '/var/scratch/lombardo/download_PS/masks_resized_100_sidewalk_masks/input_coco_format_100.json'
json_path = '/var/scratch/lombardo/download_PS/masks_resized_100_bboxes_fb1/input_coco_format_100.json'

with open(json_path, 'r') as f:
    data = json.load(f)

unique_panos = len(set([pano['pano_id'] for pano in data]))

print(f'Number of unique panos: {unique_panos}')

In [ ]:
def map_faces(index):
    # Function that maps the face index to the corresponding face
    # Mapping: 0F 1R 2B 3L 4U 5D
    # For example: index == 0, return 'front'
    if index == 0:
        return 'front'
    elif index == 1:
        return 'right'
    elif index == 2:
        return 'back'
    elif index == 3:
        return 'left'
    elif index == 4:
        return 'top'
    elif index == 5:
        return 'bottom'

In [ ]:
import torch
from torchvision.ops import box_iou
import pycocotools.mask as mask_util

def get_bbox(mask):
    """Function to convert a mask to a bounding box."""
    pos = np.where(mask)
    xmin = np.min(pos[1])
    xmax = np.max(pos[1])
    ymin = np.min(pos[0])
    ymax = np.max(pos[0])
    return [xmin, ymin, xmax, ymax]

def visualize(pano_id, face_idx, masks, bboxes):
    # Load image
    image_path = f'/var/scratch/lombardo/download_PS/reprojected/{pano_id}/{map_faces(face_idx)}.png'
    image = cv2.imread(image_path)
    # Load masks
    for mask_dict in masks:
        rle = mask_dict['segmentation']
        pred_mask = mask_util.decode(rle)
        pred_bbox = get_bbox(pred_mask)
        # Draw bbox. Format: [xmin, ymin, xmax, ymax]
        cv2.rectangle(image, (pred_bbox[0], pred_bbox[1]), (pred_bbox[2], pred_bbox[3]), (0, 255, 0), 2)
    # Load bboxes. Format: [xmin, ymin, xmax, ymax]
    for bbox in bboxes:
        cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 0, 0), 2)
    # Visualize
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.imshow(image)
    # Show the image
    plt.show()

def best_iou(pred_masks, bboxes):
    best_ious = []
    best_mask_indices = []
    bboxes_indices = []

    for bbox_idx, bbox in enumerate(bboxes):
        gt_bbox = torch.tensor([bbox], dtype=torch.float)

        max_iou = -1
        best_mask_idx = -1

        for mask_idx, mask_dict in enumerate(pred_masks):
            rle = mask_dict['segmentation']
            pred_mask = mask_util.decode(rle)
            pred_bbox = get_bbox(pred_mask)
            if pred_bbox is None:
                continue
            pred_bbox = torch.tensor([pred_bbox], dtype=torch.float)

            # Calculate IoU using PyTorch function
            iou = box_iou(gt_bbox, pred_bbox).item()

            if iou > max_iou:
                max_iou = iou
                best_mask_idx = mask_idx

        best_ious.append(max_iou)
        best_mask_indices.append(best_mask_idx)
        bboxes_indices.append(bbox_idx)

    return best_ious, best_mask_indices, bboxes_indices

def evaluate_single_face(masks, bboxes):
    '''Access the list of dictionaries masks for the current pano and face. The structure is the following:
    masks = [
                {
                'pano_id': 'pano_id_0',
                'face_idx': 'face_idx_0',
                'area': 'area',
                'centroid': [x, y],
                'segmentation': {
                    'size': [height, width],
                    'counts': 'RLE'
                    },
                },
                {...}, {...}, ...
            ]
    '''
    pano_id = masks[0]['pano_id']
    face_idx = masks[0]['face_idx']
    print(f"Number of bboxes: {len(bboxes)}")
    n_masks = len(masks)
    print(f'Number of masks: {n_masks}')
    
    # Compute metrics
    # IoU
    best_ious, mask_indices_iou, bboxes_indices_iou = best_iou(masks, bboxes)

    '''Structure of the output dictionary:
        results = {
                    'pano_id_0': {
                        'face_idx_0': {
                            'closest_points': {
                                'metrics': {
                                    'distances': [],
                                    'precision': [],
                                    'recall': [],
                                    'f1': [],
                                    'average_precision': [],
                                },
                                'points': [],
                                'indices': {
                                    'mask_indices': [],
                                    'gt_indices': []
                                }
                            },
                            'anchor_points': {
                                'metrics': {
                                    'distances': [],
                                    'precision': [],
                                    'recall': [],
                                    'f1': [],
                                    'average_precision': [],
                                },
                                'points': [],
                                'indices': {
                                    'mask_indices': [],
                                    'gt_indices': []
                                }
                            },
                            'iou_metrics': {
                                'best_ious': [],
                                'ap_50': [],
                                'ap_75': []
                            },
                            'iou_indices': {
                                'mask_indices': [],
                                'gt_indices': []
                            }
                        },
                        'face_idx_1': {...},
                    },
                    'pano_id_1': {...}
                }'''
    # Return the metrics, points, and indices
    return best_ious, mask_indices_iou, bboxes_indices_iou

In [ ]:
from tqdm import tqdm
import os
import json
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load masks from .json file
'''The panos_info dictionary will have the following structure:
    panos_info = [
            {
            'pano_id': {
                'face_idx': {
                    'masks': [...],
                    'areas': [...],
                    'centroids': [...]
                },
                ...
            },
            ...
        }
    ]'''
json_path = os.path.join('/var/scratch/lombardo/download_PS/masks_resized_100', 'input_coco_format_100.json')
with open(json_path) as f:
    panos_info = json.load(f)

# Load bboxes from .csv file
bboxes = pd.read_csv(os.path.join('/var/scratch/lombardo/download_PS/bboxes.csv'))

# Count number of unique panos (pano is a dictionary)
pano_ids = set(pano['pano_id'] for pano in panos_info)
num_panoramas = len(pano_ids)
print(f'Number of panos: {num_panoramas}')
# Count number of masks (panos_info['pano_id']['face_idx']['masks'])
print(f'Number of masks: {len(panos_info)}')
# Count number of panos with bounding boxes
bboxes_annotations = os.listdir('/var/scratch/lombardo/download_PS/annotations')
print(f'Number of bounding boxes: {len(bboxes_annotations)}')

'''The structure of grouped_panos will be:
grouped_panos = {
    'pano_id_0': {
        'face_idx_0': [
            {
                'pano_id': 'pano_id_0',
                'face_idx': 'face_idx_0',
                'area': 'area',
                'centroid': [x, y],
                'segmentation': {
                    'size': [height, width],
                    'counts': 'RLE'
                    },
            },
            {...},
            ...
        ]
        'face_idx_1': [...],
        ...
    },
    'pano_id_1': {}
    ...
}'''
grouped_panos = {}
for pano in panos_info:
    pano_id = pano['pano_id']
    face_idx = pano['face_idx']

    if pano_id not in grouped_panos:
        grouped_panos[pano_id] = {}

    if face_idx not in grouped_panos[pano_id]:
        grouped_panos[pano_id][face_idx] = []

    grouped_panos[pano_id][face_idx].append(pano)

# Evaluate each panorama's masks
# Save results in a dictionary
results = {} # for each face
mean_results = {} # for each pano

empty_panos = []

pano_counter = 0
for pano_id in tqdm(grouped_panos):

    if pano_id not in results:
        results[pano_id] = {}

    if pano_id in bboxes_annotations:
        print(f'Processing pano {pano_id}:')
        for face_idx in grouped_panos[pano_id]:
            print(f'Number of masks for face {face_idx}: {len(grouped_panos[pano_id][face_idx])}')
            # Print number of bboxes for this pano and face
            # The structure of bboxes is a PD dataframe with the following columns: pano_id, face_idx, bboxes
            # The column bboxes contains a list of bboxes for each pano: [[xmin,ymin,xmax,ymax], [], ...]
            bboxes_face = bboxes[(bboxes['pano_id'] == pano_id) & (bboxes['face_idx'] == face_idx)]['bboxes'].values[0]
            # The problem is that bboxes_face is a string, not a list
            # Convert it to a list
            bboxes_face = eval(bboxes_face)
            # Convert the string inside the list to float
            bboxes_face = [[int(float(x)) for x in bbox] for bbox in bboxes_face]
            print(f'Number of bboxes for face {face_idx}: {len(bboxes_face)}')

            if len(bboxes_face) == 0:
                print('The face does not contain obstacles.')
                empty_panos.append((pano_id, face_idx))
                break

            for bbox in bboxes_face:
                print(bbox)
            
            masks = grouped_panos[pano_id][face_idx]

            if face_idx not in results[pano_id]:
                results[pano_id][face_idx] = {}

            a, b, c = evaluate_single_face(masks, bboxes_face)
            print(f'Best IoU: {a}')
            print(f'Mask Indices IoU: {b}')
            print(f'Bboxes Indices IoU: {c}')

            visualize(pano_id, face_idx, masks, bboxes_face)

            pano_counter += 1
            if pano_counter == 1:
                break
    if pano_counter == 1:
        break
            

# Prepare pipeline visualization

In [ ]:
import json
import os
import geopandas as gpd

# Load masks
mask_path_baseline = '/var/scratch/lombardo/download_PS/masks_resized_100'
mask_path_blacken = '/var/scratch/lombardo/download_PS/masks_blackened_100_resized_100'
mask_path_bboxes = '/var/scratch/lombardo/download_PS/masks_resized_100_bboxes_fb1'
mask_path_sidewalkmasks = '/var/scratch/lombardo/download_PS/masks_resized_100_sidewalk_masks_mind100_fb1'

json_path_baseline = os.path.join(mask_path_baseline, 'input_coco_format_100.json')
json_path_blacken = os.path.join(mask_path_blacken, 'input_coco_format_100.json')
json_path_bboxes = os.path.join(mask_path_bboxes, 'input_coco_format_100.json')
json_path_sidewalkmasks = os.path.join(mask_path_sidewalkmasks, 'input_coco_format_100.json')

reprojected_blackened_path = '/var/scratch/lombardo/download_PS/reprojected_blackened_100'

In [ ]:
def load_panos_info(json_path):

    with open(json_path) as f:
        panos_info = json.load(f)

    # Count number of unique panos (pano is a dictionary)
    pano_ids = set(pano['pano_id'] for pano in panos_info)
    num_panoramas = len(pano_ids)
    print(f'Number of panos: {num_panoramas}')
    # Count number of masks (panos_info['pano_id']['face_idx']['masks'])
    print(f'Number of masks: {len(panos_info)}')

    '''The structure of grouped_panos will be:
    grouped_panos = {
        'pano_id_0': {
            'face_idx_0': [
                {
                    'pano_id': 'pano_id_0',
                    'face_idx': 'face_idx_0',
                    'area': 'area',
                    'centroid': [x, y],
                    'segmentation': {
                        'size': [height, width],
                        'counts': 'RLE'
                        },
                },
                {...},
                ...
            ]
            'face_idx_1': [...],
            ...
        },
        'pano_id_1': {}
        ...
    }'''
    grouped_panos = {}
    for pano in panos_info:
        pano_id = pano['pano_id']
        face_idx = pano['face_idx']

        if pano_id not in grouped_panos:
            grouped_panos[pano_id] = {}

        if face_idx not in grouped_panos[pano_id]:
            grouped_panos[pano_id][face_idx] = []

        grouped_panos[pano_id][face_idx].append(pano)

    return grouped_panos

In [ ]:
# Load all masks
grouped_panos_baseline = load_panos_info(json_path_baseline)
grouped_panos_blacken = load_panos_info(json_path_blacken)
grouped_panos_bboxes = load_panos_info(json_path_bboxes)
grouped_panos_sidewalkmasks = load_panos_info(json_path_sidewalkmasks)

In [ ]:
import os
import json
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random

reprojected_folder = '/var/scratch/lombardo/download_PS/reprojected'
segmented_folder = '/var/scratch/lombardo/download_PS/segmented_B_0.25'
algorithm_mapping = {
    'baseline': grouped_panos_baseline,
    'blacken': grouped_panos_blacken,
    'bboxes': grouped_panos_bboxes,
    'sidewalkmasks': grouped_panos_sidewalkmasks
}

# Randomize the order of the panos
pano_ids = list(algorithm_mapping['baseline'].keys())  # Assumes same keys across all algorithms
np.random.shuffle(pano_ids)
for pano in pano_ids:
    pano_id = pano
    #pano_id = '_pjTmksYX5Vpgoqm_GSOKA'
    #pano_id = '4w05JbqOnGLYHAt6zKDbZQ'
    #pano_id = '0m9Or3xbo7JvO34d39YHSg'
    #pano_id = '-NVzemAJVgKxPc5icSUDxw'
    print(f'Pano: {pano_id}')

    for face in ['front', 'back', 'left', 'right']:
        #face = 'back'
        #face = 'right'
        #face = 'back'
        face = 'front'
        print(f'Face: {face}')

        # Check if sidewalk mask exists
        bboxes_path = os.path.join(segmented_folder, pano_id, f'mask_{face}.json')
        with open(bboxes_path) as f:
                bboxes_data = json.load(f)
        exists_flag = False
        for bbox_d in bboxes_data:
            if bbox_d.get('label') == 'sidewalk':
                exists_flag = True
                break
        if not exists_flag:
            print('No sidewalk label found')
            continue
        else:


            for algorithm in ['baseline', 'blacken', 'bboxes', 'sidewalkmasks']:
                print(f'Algorithm: {algorithm}')
                grouped_panos = algorithm_mapping[algorithm]

                # Find the reprojected image in the reprojected folder
                if algorithm == 'blacken':
                    image_path = os.path.join(reprojected_blackened_path, pano_id, face + '.png')
                else:
                    image_path = os.path.join(reprojected_folder, pano_id, face + '.png')
                
                image = cv2.imread(image_path)
                # Convert to RGB
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                # Load the semantic segmentation results in segmented_folder/pano_id/sidewalk_mask_{face}.jpg'
                sidewalk_mask_path = os.path.join(segmented_folder, pano_id, 'sidewalk_mask_' + face + '.jpg')
                # Open the sidewalk mask
                size = 1024
                sidewalk_mask = cv2.imread(sidewalk_mask_path, cv2.IMREAD_GRAYSCALE)
                sidewalk_mask = cv2.resize(sidewalk_mask, (size, size))
                _, sidewalk_mask = cv2.threshold(sidewalk_mask, 127, 255, cv2.THRESH_BINARY)

                # Prepare overlay with semantic segmentation results
                semseg_image_copy = image.copy()
                # Overlay sidewalk mask and bbox with transparency of 0.5
                semseg_image_copy[sidewalk_mask == 255] = (semseg_image_copy[sidewalk_mask == 255] * 0.5 + np.array([255, 255, 255]) * 0.5).astype(np.uint8)

                # Initialize bbox_mask
                bbox_mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
                for bbox_d in bboxes_data:
                    #if bbox_d.get('label') in ['sidewalk', 'sidewalkroad']:
                    if bbox_d.get('label') in ['sidewalk']:
                        print(bbox_d.get('label'))
                        bbox = bbox_d.get('box')
                        cv2.rectangle(bbox_mask, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), 255, 2)
                        cv2.rectangle(semseg_image_copy, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 160, 0), 3)

                # First for white masks, second for colored masks
                #object_masks = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
                object_masks = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
                
                face_idx = reverse_map_faces(face)

                for masks_face_idx, masks_list in grouped_panos[pano_id].items():
                    if masks_face_idx == int(face_idx):
                        for idx, mask_dict in enumerate(masks_list):
                            rle = mask_dict['segmentation']
                            mask = mask_util.decode(rle)
                            color = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
                            #object_masks[mask == 1] = 255
                            object_masks[mask == 1] = color

                # Image copy
                image_copy = image.copy()
                # Overlay object_masks with transparency of 0.3
                image_copy[object_masks == 255] = (image_copy[object_masks == 255] * 0.5 + np.array([255, 255, 255]) * 0.5).astype(np.uint8)

                # Visualize the image, the sidewalk mask and the bbox mask using matplotlib
                #fig, ax = plt.subplots(1, 5, figsize=(15, 15))
                fig, ax = plt.subplots(1, 3, figsize=(15, 15))
                ax[0].imshow(image)
                #ax[1].imshow(sidewalk_mask)
                ax[1].imshow(semseg_image_copy)
                #ax[2].imshow(bbox_mask)
                #ax[3].imshow(object_masks)
                #ax[4].imshow(image_copy)
                ax[2].imshow(image_copy)
                for a in ax:
                    a.set_xticks([])
                    a.set_yticks([])
                plt.show()

                # Save everything
                #if algorithm == 'blacken':
                    #cv2.imwrite(f'res/pipeline/{pano_id}_{face}_reprojected.png', cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                #cv2.imwrite(f'res/pipeline/{pano_id}_{face}_{algorithm}_semseg.png', cv2.cvtColor(semseg_image_copy, cv2.COLOR_BGR2RGB))
                #cv2.imwrite(f'res/pipeline/{pano_id}_{face}_{algorithm}_results.png', cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB))
            break
    break

In [ ]:
# Create input images for pipeline visualization

# Load image from 'res/equirectangular.png'
reoriented_folder = '/var/scratch/lombardo/download_PS/reoriented'
img_path = os.path.join(reoriented_folder, '-NVzemAJVgKxPc5icSUDxw.jpg')
reoriented_img = cv2.imread(img_path)

# Convert to cubemap
cubemap = py360convert.e2c(reoriented_img, face_w=1024)
# Print dtype of cubemap
print(cubemap.dtype)
# Flip horizontally the images of cubemap at indices 0 and 2
#cubemap[[0,2]] = cubemap[[0,2]][:,::-1]
# Save cubemap as example_pano_triangle_cb.png
cv2.imwrite('res/pipeline_reoriented_cubemap.jpg', cubemap)

# Visualize Algorithms

In [ ]:
def visualize_labels(args, faces, pano_id, masks, labels_df, directory):

    for face_idx in faces:
        # Reverse map the face index to the face name
        face = map_faces(int(face_idx))

        # Check if we already processed this face
        output_path = os.path.join(directory, f'visualize_{pano_id}_{face}.png')
        if os.path.exists(output_path):
            continue

        fig = plt.figure()
        # Load the face
        face_path = os.path.join(args.input_dir, pano_id, f'{face}.png')
        img = cv2.imread(face_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)

        # Traverse the masks in masks_list, decode them and load each of them with a different color
        output_mask = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

        for masks_face_idx, masks_list in masks.items():
            if masks_face_idx == int(face_idx):
                for idx, mask_dict in enumerate(masks_list):
                    rle = mask_dict['segmentation']
                    mask = mask_util.decode(rle)
                    color = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
                    output_mask[mask == 1] = color

        # Plot the mask
        plt.imshow(output_mask, alpha=0.4)
        #mask_path = os.path.join(args.masks_dir, pano_id, f'{face}.png')
        #mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        #plt.imshow(mask, alpha=0.5, cmap='gray')

        # Load ground truth points knowing that the coordinates are (reprojected_pano_y,reprojected_pano_x)
        gt_points = []
        for idx, row in labels_df.iterrows():
                if row['gsv_panorama_id'] == pano_id.replace("'", "") and row['face_idx'] == int(face_idx):
                    gt_points.append([row['reprojected_pano_y'], row['reprojected_pano_x']])

        for gt_point in gt_points:
            plt.scatter(gt_point[1], gt_point[0], color='red', s=7)

        # Set small tick labels
        plt.tick_params(axis='both', which='both', labelsize=6)

        # Add legend for red points
        plt.scatter([], [], color='red', s=8, label='Ground Truth')
        plt.legend(prop={'size': 5})

        # Save the image
        fig.savefig(os.path.join(directory, f'visualize_{pano_id}_{face}.png'), dpi=300, bbox_inches='tight')
        # Close the figure
        plt.close(fig)

In [ ]:
import os
import json
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random

reprojected_folder = '/var/scratch/lombardo/download_PS/reprojected'
segmented_folder = '/var/scratch/lombardo/download_PS/segmented_B_0.25'
algorithm_mapping = {
    'baseline': grouped_panos_baseline,
    'blacken': grouped_panos_blacken,
    'bboxes': grouped_panos_bboxes,
    'sidewalkmasks': grouped_panos_sidewalkmasks
}

def save_subplot(ax, filename):
    fig = plt.figure()
    new_ax = fig.add_subplot(111, frame_on=False)
    new_ax.xaxis.set_ticks([])
    new_ax.yaxis.set_ticks([])
    new_ax.set_xticklabels([])
    new_ax.set_yticklabels([])
    for i in range(len(ax.images)):
        new_ax.imshow(ax.images[i].get_array(), alpha=ax.images[i].get_alpha(), cmap=ax.images[i].get_cmap())
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close(fig)

# Randomize the order of the panos
pano_ids = list(algorithm_mapping['baseline'].keys())  # Assumes same keys across all algorithms
np.random.shuffle(pano_ids)
for pano in pano_ids:
    #pano_id = pano
    #pano_id = '_pjTmksYX5Vpgoqm_GSOKA'
    #pano_id = '4w05JbqOnGLYHAt6zKDbZQ'
    #pano_id = '0m9Or3xbo7JvO34d39YHSg'
    #pano_id = '-NVzemAJVgKxPc5icSUDxw'
    #pano_id = '_IkbLqsPGpvTl3VC_QAugw'
    pano_id = 'qHGj7xzo1LgCxDXyjtP_0g'
    print(f'Pano: {pano_id}')

    # Print the row of labels_reprojected that contains the pano_id in gsv_panorama_id
    #print(labels_reprojected[labels_reprojected['gsv_panorama_id'] == pano_id.replace("'", "")])

    for face in ['front', 'back', 'left', 'right']:
        #face = 'back'
        #face = 'right'
        #face = 'back'
        #face = 'front'
        print(f'Face: {face}')

        # Check if sidewalk mask exists
        bboxes_path = os.path.join(segmented_folder, pano_id, f'mask_{face}.json')
        with open(bboxes_path) as f:
                bboxes_data = json.load(f)
        exists_flag = False
        for bbox_d in bboxes_data:
            if bbox_d.get('label') == 'sidewalk':
                exists_flag = True
                break
        if not exists_flag:
            print('No sidewalk label found')
            continue
        else:
            
            face_idx = reverse_map_faces(face)

            # Load ground truth labels
            gt_points = []
            for idx, row in labels_reprojected.iterrows():
                    if row['gsv_panorama_id'] == pano_id.replace("'", "") and row['face_idx'] == int(face_idx):
                        gt_points.append([row['reprojected_pano_y'], row['reprojected_pano_x']])

            #print(f'Number of gt_points: {len(gt_points)}')
            #if len(gt_points) == 0:
            #    break

            # Find the reprojected image in the reprojected folder
            image_path = os.path.join(reprojected_folder, pano_id, face + '.png')
            image = cv2.imread(image_path)

            # Convert to RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # Load the semantic segmentation results in segmented_folder/pano_id/sidewalk_mask_{face}.jpg'
            sidewalk_mask_path = os.path.join(segmented_folder, pano_id, 'sidewalk_mask_' + face + '.jpg')
            # Open the sidewalk mask
            size = 1024
            sidewalk_mask = cv2.imread(sidewalk_mask_path, cv2.IMREAD_GRAYSCALE)
            sidewalk_mask = cv2.resize(sidewalk_mask, (size, size))
            _, sidewalk_mask = cv2.threshold(sidewalk_mask, 127, 255, cv2.THRESH_BINARY)

            # Prepare overlay with semantic segmentation results
            semseg_image_copy = image.copy()
            # Overlay sidewalk mask and bbox with transparency of 0.5
            #semseg_image_copy[sidewalk_mask == 255] = (semseg_image_copy[sidewalk_mask == 255] * 0.5 + np.array([255, 255, 255]) * 0.5).astype(np.uint8)

            semseg_mask = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
            color = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
            #object_masks[mask == 1] = 255
            semseg_mask[sidewalk_mask == 255] = color

            # Initialize bbox_mask
            bbox_mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
            for bbox_d in bboxes_data:
                #if bbox_d.get('label') in ['sidewalk', 'sidewalkroad']:
                if bbox_d.get('label') in ['sidewalk']:
                    print(bbox_d.get('label'))
                    bbox = bbox_d.get('box')
                    cv2.rectangle(bbox_mask, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), 255, 2)
                    cv2.rectangle(semseg_image_copy, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 160, 0), 3)

            # Initialize figure
            fig, ax = plt.subplots(1, 5, figsize=(15, 15))

            # Show input
            ax[0].imshow(image)
            # Show semantic segmentation
            ax[1].imshow(semseg_image_copy)
            ax[1].imshow(semseg_mask, alpha=0.4)

            if face == 'right':
                save_subplot(ax[0], f'res/experiments/{pano_id}_{face}_0.png')
                save_subplot(ax[1], f'res/experiments/{pano_id}_{face}_1.png')

            counter = 2
            for algorithm in ['baseline', 'bboxes', 'sidewalkmasks']:
                print(f'Algorithm: {algorithm}')
                grouped_panos = algorithm_mapping[algorithm]

                # First for white masks, second for colored masks
                #object_masks = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
                object_masks = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)

                for masks_face_idx, masks_list in grouped_panos[pano_id].items():
                    if masks_face_idx == int(face_idx):
                        for idx, mask_dict in enumerate(masks_list):
                            rle = mask_dict['segmentation']
                            mask = mask_util.decode(rle)
                            color = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
                            #object_masks[mask == 1] = 255
                            object_masks[mask == 1] = color

                # Image copy
                image_copy = image.copy()
                # Overlay object_masks with transparency of 0.3
                #image_copy[object_masks == 255] = (image_copy[object_masks == 255] * 0.5 + np.array([255, 255, 255]) * 0.5).astype(np.uint8)

                # Visualize the image, the sidewalk mask and the bbox mask using matplotlib
                #fig, ax = plt.subplots(1, 5, figsize=(15, 15))
                
                
                #ax[2].imshow(bbox_mask)
                #ax[3].imshow(object_masks)
                #ax[4].imshow(image_copy)
                ax[counter].imshow(image_copy)
                ax[counter].imshow(object_masks, alpha=0.4)
                
                if face == 'right':
                    save_subplot(ax[counter], f'res/experiments/{pano_id}_{face}_{algorithm}_{counter}.png')
                
                counter += 1

            for a in ax:
                a.set_xticks([])
                a.set_yticks([])
                '''for gt_point in gt_points:
                    a.scatter(gt_point[1], gt_point[0], color='red', s=7)
                    # Add legend for red points
                    a.scatter([], [], color='red', s=8, label='Ground Truth')
                    a.legend(prop={'size': 5})'''
            plt.show()
                   
    break

# Differentiate performance by neighbourhood (Stadsdelen)

In [ ]:
import http.client
import json
import requests

def fetch_pano_ids_from_webserver():
    unique_ids = set()
    pano_info = []
    sidewalk_server_fqdn = "sidewalk-amsterdam.cs.washington.edu"
    conn = http.client.HTTPSConnection(sidewalk_server_fqdn)
    conn.request("GET", "/adminapi/panos")
    r1 = conn.getresponse()
    data = r1.read()
    jsondata = json.loads(data)

    # Structure of JSON data
    # [
    #     {
    #         "gsv_panorama_id": "example-id",
    #         "image_width": 16384,
    #         "image_height": 8192,
    #         "panorama_lat": 52.315662,
    #         "panorama_lng":4.9531154,
    #         "photographer_heading":1.0355758,
    #         "photographer_pitch":240.7730102
    #     },
    #     ...
    # ]
    for value in jsondata:
        pano_id = value["gsv_panorama_id"]
        if pano_id not in unique_ids:
            # Check if the pano_id is an empty string.
            if pano_id and pano_id != 'tutorial':
                unique_ids.add(pano_id)
                pano_info.append(value)
            else:
                print("Pano ID is an empty string or is for tutorial")
        else:
            print("Duplicate pano ID")
    assert len(unique_ids) == len(pano_info)
    #print("Number of unique pano IDs: {}".format(len(unique_ids)))
    #print(pano_info[:10])
    return pano_info

pano_info = fetch_pano_ids_from_webserver()
print(len(pano_info))

regions = requests.get('https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=INDELING_STADSDEEL&THEMA=gebiedsindeling').json()
regions_df = gpd.GeoDataFrame.from_features(regions['features'])

In [ ]:
# pano_info is a list of dictionaries with the following structure:
# [
#     {
#         "gsv_panorama_id": "example-id",
#         "width": 16384,
#         "height": 8192,
#         "lat": 52.315662,
#         "lng": 4.9531154,
#         ...
#     },
#     ...
# ]
# Filter from pano_info the pano_ids (gsv_panorama_id) that are NOT in the list of pano_ids
filtered_pano_info = [p for p in pano_info if p['gsv_panorama_id'] in pano_ids]
print(len(filtered_pano_info))
# Make a dataframe out of gsv_panorama_id and lat, lng
neighbourhood_df = pd.DataFrame(filtered_pano_info)[['gsv_panorama_id', 'lat', 'lng']]
# Convert the dataframe to a geodataframe with a Point geometry POINT (lng lat)
neighbourhood_gdf = gpd.GeoDataFrame(neighbourhood_df, geometry=gpd.points_from_xy(neighbourhood_df.lng, neighbourhood_df.lat))
panos_with_neighbourhoods = gpd.sjoin(neighbourhood_gdf, regions_df)

In [ ]:
# Add Stadsdeel to the results of different algorithms
baseline_results_path = '/var/scratch/lombardo/download_PS/evaluation_100_100_100/mean_results.csv'
blacken_results_path = '/var/scratch/lombardo/download_PS/evaluation_blackened_100_100_100_100/mean_results.csv'
bboxes_results_path = '/var/scratch/lombardo/download_PS/evaluation_bboxes_100_100_100_fb1/mean_results.csv'
sidewalkmasks_results_path = '/var/scratch/lombardo/download_PS/evaluation_masks_100_100_100_mind100_fb1/mean_results.csv'

baseline_results = pd.read_csv(baseline_results_path)
blacken_results = pd.read_csv(blacken_results_path)
bboxes_results = pd.read_csv(bboxes_results_path)
sidewalkmasks_results = pd.read_csv(sidewalkmasks_results_path)

In [ ]:
# For each dataframe, add the column 'Stadsdeel' with the corresponding value (gsv_panorama_id == pano_id)
baseline_results['Stadsdeel'] = baseline_results['pano_id'].map(panos_with_neighbourhoods.set_index('gsv_panorama_id')['Stadsdeel'])
blacken_results['Stadsdeel'] = blacken_results['pano_id'].map(panos_with_neighbourhoods.set_index('gsv_panorama_id')['Stadsdeel'])
bboxes_results['Stadsdeel'] = bboxes_results['pano_id'].map(panos_with_neighbourhoods.set_index('gsv_panorama_id')['Stadsdeel'])
sidewalkmasks_results['Stadsdeel'] = sidewalkmasks_results['pano_id'].map(panos_with_neighbourhoods.set_index('gsv_panorama_id')['Stadsdeel'])

# For each dataframe, group by Stadsdeel and compute the mean of the columns (excluding pano_id and stadsdeel)
baseline_results_grouped = baseline_results.groupby('Stadsdeel').mean().reset_index()
blacken_results_grouped = blacken_results.groupby('Stadsdeel').mean().reset_index()
bboxes_results_grouped = bboxes_results.groupby('Stadsdeel').mean().reset_index()
sidewalkmasks_results_grouped = sidewalkmasks_results.groupby('Stadsdeel').mean().reset_index()

# Save in the corresponding path, a new csv file with the results grouped by Stadsdeel
baseline_results_grouped.to_csv('/var/scratch/lombardo/download_PS/evaluation_100_100_100/mean_results_neighbourhood.csv', index=False)
blacken_results_grouped.to_csv('/var/scratch/lombardo/download_PS/evaluation_blackened_100_100_100_100/mean_results_neighbourhood.csv', index=False)
bboxes_results_grouped.to_csv('/var/scratch/lombardo/download_PS/evaluation_bboxes_100_100_100_fb1/mean_results_neighbourhood.csv', index=False)
sidewalkmasks_results_grouped.to_csv('/var/scratch/lombardo/download_PS/evaluation_masks_100_100_100_mind100_fb1/mean_results_neighbourhood.csv', index=False)


# Print how many panoramas are in each Stadsdeel
print(baseline_results['Stadsdeel'].value_counts())

In [ ]:
# Show 'mean_cp_precision', 'mean_cp_recall', 'mean_cp_f1', 'mean_cp_ap' for each Stadsdeel of baseline_results_grouped
print(baseline_results_grouped[['Stadsdeel', 'mean_cp_precision', 'mean_cp_recall', 'mean_cp_f1', 'mean_cp_ap']])

# Show 'mean_cp_precision', 'mean_cp_recall', 'mean_cp_f1', 'mean_cp_ap' for each Stadsdeel of blacken_results_grouped
print(blacken_results_grouped[['Stadsdeel', 'mean_cp_precision', 'mean_cp_recall', 'mean_cp_f1', 'mean_cp_ap']])

# Show 'mean_cp_precision', 'mean_cp_recall', 'mean_cp_f1', 'mean_cp_ap' for each Stadsdeel of bboxes_results_grouped
print(bboxes_results_grouped[['Stadsdeel', 'mean_cp_precision', 'mean_cp_recall', 'mean_cp_f1', 'mean_cp_ap']])

# Show 'mean_cp_precision', 'mean_cp_recall', 'mean_cp_f1', 'mean_cp_ap' for each Stadsdeel of sidewalkmasks_results_grouped
print(sidewalkmasks_results_grouped[['Stadsdeel', 'mean_cp_precision', 'mean_cp_recall', 'mean_cp_f1', 'mean_cp_ap']])

In [ ]:
# Print 10 gsv_panorama_ids from different Stadsdeel

path = '/var/scratch/lombardo/download_PS/reoriented/'

# Open them as images
# Make a fig with 10 images, 5 per row
fig, ax = plt.subplots(2, 5, figsize=(20, 8))
for i, pano_id in enumerate(bboxes_results[bboxes_results['Stadsdeel'] == 'Centrum']['pano_id'].sample(10)):
    pano = cv2.imread(os.path.join(path, pano_id + '.jpg'), cv2.IMREAD_COLOR)
    ax[i//5, i%5].imshow(cv2.cvtColor(pano, cv2.COLOR_BGR2RGB))
    ax[i//5, i%5].set_title(pano_id)
    ax[i//5, i%5].axis('off')
    # Tight layout
    fig.tight_layout()
plt.show()

In [ ]:
# Make a fig with 10 images, 5 per row
fig, ax = plt.subplots(2, 5, figsize=(20, 8))
for i, pano_id in enumerate(bboxes_results[bboxes_results['Stadsdeel'] == 'Noord']['pano_id'].sample(10)):
    pano = cv2.imread(os.path.join(path, pano_id + '.jpg'), cv2.IMREAD_COLOR)
    ax[i//5, i%5].imshow(cv2.cvtColor(pano, cv2.COLOR_BGR2RGB))
    ax[i//5, i%5].set_title(pano_id)
    ax[i//5, i%5].axis('off')
    # Tight layout
    fig.tight_layout()
plt.show()

# Count number of problems in semantic segmentation

In [ ]:
# Define the subset of the dataset used in the qualitative evaluation (84 panoramas)
pano_ids = {'0FHr9qesIVgabJ8hR7Uf2A', '1-TT8bGW5yAUXJKm40xrJQ', '1GIXHFehrd1Xbu2JbmzlFw', '1vSh2gNp3jJsrd1G_sl-BQ', '2atLrFYpUNAyUc7f7M8mzA', '3iQ5g0gAW8uyiTwiRNytaQ',
            '2Yg6fuCTqr_5Uc5IcGPs4w', '0X1EzeV1Gae6VgvEonwc6w', '-juZj6rt9x1unIRBa0D7fQ', '2eQKbDNhkY-sD01QunNhjg', '2qShzHuplxE5Zp02V3Fnrw', '2uwdgegUMu_oMh5vILMtqg', 
            '1VO2N7ibSMEKWX0SAL4DoQ', '-RHS-ZYDt0sCDy2aS4AwOQ', '2fPjt5siCmoOpY1kMe3t3w', '_mx7T3kIbcEWe0NF28ohoQ', '-ELBA4Cjr_WBoetht0GVbQ', '2ZRURFi8Z58Od7a0tyK7oQ', 
            '0wLMkTJ7IaDz8mJm3uNeHg', '0sKrgbDBfDrizWjCnXI0FQ', '_Nkwp741bbV44txP6PRo1A', '2iKOp2S2_K4WrxfLS1jDlA', '0xS8FZwZ3ueZuMoD1Okywg', '1ex08CPMxpK3rK38W5gmRA', 
            '-ay7zopCWOrSjeUuirk91A', '-paSEQ-OawExso6Za-T-ug', '1VEVLWYfmi0vVyEsXoJD8A', '2E3r3TggV5YhVAYv1q0euA', '3BronM9R13wOkieBgnTV5w', '0yCv0ur6psBphmog_DOTOQ', 
            '4XfWRDhr6JF0kaotjzOKeQ', '0WuvMRCqZvf5e_unk3EG3Q', '3d2ja9krEsxrAcl1UYSH6g', '0KsmEkVuSWOERkQZ6ZgF-w', '5CIoi-u3-z4tCk5-5MXpxw', '3R9FNgg-dKle9aW5LsbnXQ', 
            '2EoZiLmHGl61m3r1wp4x8w', '1GYNc35brG1DJCtFYWs8KA', '0m9Or3xbo7JvO34d39YHSg', '4mw9TKPCSU7UbOpj55JGtw', '1KVPOen47WIgzsK0mMohNg', '1WQkqFs-U-85F5TzNZ6smQ', 
            '0E_M8eWvyACpG_fUq4TFrQ', '1sn9J4pmBrGqPClank2kiw', '-75fMXMEzhJHhe7FBqSRAw', '1sMhc-pYJPzUCZi40ZzDqg', '-WI1nKjShmeFVRGsrFsFpw', '_EHBpNS1aJR8rCWf_7a93A', 
            '0h2CfQQ_Ca5sRgU5Zze01g', '2ke_0eGW8-nEpc1cFyBM2A', '0jukKJk2AKvpFR7l88fHUA', '4w05JbqOnGLYHAt6zKDbZQ', '2uAdFMxVC3otCoJUJNaxmw', '3R5bt2FGbk3HWXKuUxno8w', 
            '_A3VGaMhL4XcGQxtFUiAfA', '4hVg_tOU2RRNlx_t1Zl6Lw', '3csQYBZY-qDW7hWrCf89zw', '_-7gKmiV-npQwLy_CD3v7Q', '0NjLmtmwNh95YlOyyydtnA', '2FdyeM1CVFiFKeDzQCHTkw', 
            '_WZ_Q2QCOe9NEypucJnClQ', '2ZM6bXf8ZZvdpsddMHIS1Q', '-NVzemAJVgKxPc5icSUDxw', '-rU6BN-wNeRclahZxn2nug', '_ZuTw7GC38rShAUjL-dLmQ', '2pgXiZXvZ8nqablHtAOL_A', 
            '2-VGJOr3z6Xq0PZjn-3HIg', '-qi4eywPKVd0sHlHqcWVkA', '0sAG8-ozIw7wANw_hsTCHA', '_C1U2haxCCmEyOsiC_PQ2w', '-omjTAq5SaYYZWZEPmFqbA', '-I6Ikjopy1WKUGc3RRueLA', 
            '-CD3qZk9zyeumBU_w7Yl5A', '_YIVijia4obgMONc9a-Vsg', '4Wy_JKfQ0yKteDeiHg2TxQ', '3FqdF-LCwp9hyf2V6V2YEg', '_pjTmksYX5Vpgoqm_GSOKA', '0vrKe1xEAZkxUXYhtwRtaQ', 
            '5aikoK5Fm_l-jopIV0n9LQ', '4wDHo4_KCawwsEv8tHX_Qw', '3T1QwrFPgOoYScEgEjJONg', '5f5MCY9UOi1m1W2ppnHNXw', '4n3TQoipOOfVc7HyGYooUQ', '4Z0NGEN7MFBphGkxa0Wpmw'}

seg_path = '/var/scratch/lombardo/download_PS/segmented_B_0.25'
output_dir = '/var/scratch/lombardo/download_PS/seg_analysis_B_0.25'

In [ ]:
if output_dir not in os.listdir('/var/scratch/lombardo/download_PS'):
    os.mkdir(output_dir)

# For each pano id in pano_ids, take the four 'grounded_sam_output_{face}.jpg', copy in output_dir, rename in '{pano_id}_grounded_sam_output_{face}.jpg', and then go to the next pano
# face in ['front', 'back', 'left', 'right']
for pano_id in pano_ids:
    for face in ['front', 'back', 'left', 'right']:
        src = os.path.join(seg_path, pano_id, f'grounded_sam_output_{face}.jpg')
        dst = os.path.join(output_dir, f'{pano_id}_grounded_sam_output_{face}.jpg')
        shutil.copyfile(src, dst)